##### Functions to import data from Salesforce and build a Pandas dataframe

In [1]:
import pandas as pd 
from datetime import datetime
import beatbox

In [2]:
def extract_data_from_sf(sf_username, sf_password, sf_api_token):
    #Returns a pandas dataframe with all opportunities
    sf_creds=sf_password+sf_api_token
    service = beatbox.PythonClient()  
    service.login(sf_username, sf_creds)
    query_result = service.query("SELECT Id, AccountId, Name, Description, Date_d_ouverture__c, Market_Competitors__c, Business_Model_Figures__c, Resume__c, Team__c, Round__c, Amount, CurrencyIsoCode, LeadSource, X1st_meeting__c FROM Opportunity WHERE Date_d_ouverture__c > 2009-01-01") 
    records = query_result['records']
    total_records = query_result['size']  # full size of results
    query_locator = query_result['queryLocator']  # get the mystical queryLocator
    # loop through, pulling the next 500 and appending it to your records dict
    while query_result['done'] is False and len(records) < total_records:
        query_result = service.queryMore(query_locator)
        query_locator = query_result['queryLocator']  # get the updated queryLocator
        records = records + query_result['records']
    df = pd.DataFrame(records)
    df.columns = ['AccountId', 'Amount_local_currency', 'BM_Figures', 'Currency', 'Open_Date', 'Description', 'Id', 'LeadSource', 'Market_Competitors', 'Name', 'Summary', 'Round', 'Team', '1stmeeting', 'type']
    print "Number of extracted opportunities:"
    print len(df)
    return df

In [3]:
def extract_topicassignment_from_sf(sf_username, sf_password, sf_api_token):
    sf_creds=sf_password+sf_api_token
    service = beatbox.PythonClient()  
    service.login(sf_username, sf_creds)
    query_result_topics1 = service.query("SELECT TopicId, EntityId, IsDeleted FROM TopicAssignment WHERE  IsDeleted = False") 
    records_topics1 = query_result_topics1['records']
    total_records_topics1 = query_result_topics1['size']  # full size of results
    query_locator_topics1 = query_result_topics1['queryLocator']  # get the mystical queryLocator
    # loop through, pulling the next 500 and appending it to your records dict
    while query_result_topics1['done'] is False and len(records_topics1) < total_records_topics1:
        query_result_topics1 = service.queryMore(query_locator_topics1)
        query_locator_topics1 = query_result_topics1['queryLocator']  # get the updated queryLocator
        records_topics1 = records_topics1 + query_result_topics1['records']
    TopicAssignment = pd.DataFrame(records_topics1)
    TopicAssignment = TopicAssignment[['EntityId', 'TopicId', 'IsDeleted']]
    return TopicAssignment

In [4]:
def extract_topicnames_from_sf(sf_username, sf_password, sf_api_token):
    sf_creds=sf_password+sf_api_token
    service = beatbox.PythonClient()  
    service.login(sf_username, sf_creds)
    query_result_topics2 = service.query("SELECT Id, Name FROM Topic") 
    records_topics2 = query_result_topics2['records']
    total_records_topics2 = query_result_topics2['size']  # full size of results
    query_locator_topics2 = query_result_topics2['queryLocator']  # get the mystical queryLocator
    # loop through, pulling the next 500 and appending it to your records dict
    while query_result_topics2['done'] is False and len(records_topics2) < total_records_topics2:
        query_result_topics2 = service.queryMore(query_locator_topics2)
        query_locator_topics2 = query_result_topics2['queryLocator']  # get the updated queryLocator
        records_topics2 = records_topics2 + query_result_topics2['records']
    TopicNames = pd.DataFrame(records_topics2)
    return TopicNames

In [5]:
def extract_topics_from_sf(sf_username, sf_password, sf_api_token):
    TopicAssignment = extract_topicassignment_from_sf(sf_username, sf_password, sf_api_token)
    TopicNames = extract_topicnames_from_sf(sf_username, sf_password, sf_api_token)
    topics_df = pd.merge(TopicAssignment, TopicNames, how='inner', left_on='TopicId', right_on='Id')
    topics_df.Name = topics_df.Name.map(lambda x : str(x).lower())
    return topics_df

In [6]:
import numpy as np
import re
import os
import math
np.random.seed(1234)

In [7]:
#Competitors processing
def process_competitors_and_textvariables(df):
    df['Competitors'] = 'No competitor'
    for i in range(0,len(df.index)):
        if pd.isnull(df.Market_Competitors[i]) == False:
            sub_competitor_list = []
            #Remove text between brackets
            text = re.sub("[\(\[].*?[\)\]]", "", df.Market_Competitors[i])
            #Remove multiple spaces
            text = re.sub(' +',' ',text)
            #Remove space before punctuation ("Competitors :" becomes "Competitors:")
            text = re.sub(r'\s([?.!":](?:\s|$))', r'\1', text)
            #Lower letters
            text = text.lower()
            if ("competitors:" in text) or ("competitor:" in text) or ("competition:" in text):
                competitor_words_list = text.split(' ')
                position = [competitor_words_list.index(j) for j in competitor_words_list if ('competitors:' in j) or ('competitor:' in j) or ('competition:' in j)] 
                #On rejoint tous les mots en un seul texte que l'on coupe ensuite au niveau des virgules ou des ":" ou des "+"
                sub_competitor_list = re.split(r'[,:+]+',''.join(competitor_words_list[position[0]+1:]))
            df['Competitors'][i] = sub_competitor_list
        else:
            df['Competitors'][i] = list('')
    #Process text variables
    df.BM_Figures = df.BM_Figures.map(lambda x : str(x).lower())
    df.Description = df.Description.map(lambda x : str(x).lower())
    df.Summary = df.Summary.map(lambda x : str(x).lower())
    df.Market_Competitors = df.Market_Competitors.map(lambda x : str(x).lower())
    #Create Year variable
    df['Year']= df['Open_Date'].map(lambda X: X.year)
    return df

###### Functions to check how many opportunities contain certain words

In [8]:
#Find opportunities containing certain words (list_words_to_find) and without certain words (list_words_to_avoid)

def findwordfrequency_withoutBM(df, topics_df, year_min, list_words_to_find, list_words_to_find_only_in_topics, list_words_to_avoid) :
    sub_df = df[df.Year >= year_min]
    #Search words in text
    sub_df.Wholetext = sub_df.Summary + ' ' + sub_df.Description  + ' ' + sub_df.Market_Competitors
    sub_df['check_text'] = sub_df.Wholetext.map(lambda text : 1 if any(x in text for x in list_words_to_find) else 0)
    sub_df['check_text_bis'] = sub_df.Wholetext.map(lambda text : "pb" if any(x in text for x in list_words_to_avoid) else "ok")
    sub_df.loc[sub_df['check_text_bis']=="pb", 'check_text'] = 0
    #Search words in topics
    sub_topics_df = topics_df[(topics_df.Name.isin(list_words_to_find) | topics_df.Name.isin(list_words_to_find_only_in_topics))]
    list_id_withtopics = list(set(sub_topics_df.EntityId))
    sub_df['check_topics'] = 0
    sub_df.loc[sub_df.Id.isin(list_id_withtopics), 'check_topics'] = 1
    list_id_withouttopics = list(set(topics_df[topics_df.Name.isin(list_words_to_avoid)].EntityId))
    sub_df.loc[sub_df.Id.isin(list_id_withouttopics), 'check_topics'] = 0
    #Combine both results
    sub_df['check'] =  sub_df['check_text'] + sub_df['check_topics']
    sub_df.loc[sub_df['check']>=1, 'check'] = 1
    print "Number of opportunities containing at least one of the words:"
    print len(sub_df[sub_df.check == 1])
    return sub_df


def findwordfrequency_withBM(df, topics_df, year_min, list_words_to_find, list_words_to_find_only_in_topics, list_words_to_avoid) :
    sub_df = df[df.Year >= year_min]
    #Search words in text
    sub_df.Wholetext = sub_df.Summary + ' ' + sub_df.Description  + ' ' + sub_df.Market_Competitors + ' ' + sub_df.BM_Figures
    sub_df['check_text'] = sub_df.Wholetext.map(lambda text : 1 if any(x in text for x in list_words_to_find) else 0)
    sub_df['check_text_bis'] = sub_df.Wholetext.map(lambda text : "pb" if any(x in text for x in list_words_to_avoid) else "ok")
    sub_df.loc[sub_df['check_text_bis']=="pb", 'check_text'] = 0
    #Search words in topics
    sub_topics_df = topics_df[(topics_df.Name.isin(list_words_to_find) | topics_df.Name.isin(list_words_to_find_only_in_topics))]
    list_id_withtopics = list(set(sub_topics_df.EntityId))
    sub_df['check_topics'] = 0
    sub_df.loc[sub_df.Id.isin(list_id_withtopics), 'check_topics'] = 1
    list_id_withouttopics = list(set(topics_df[topics_df.Name.isin(list_words_to_avoid)].EntityId))
    sub_df.loc[sub_df.Id.isin(list_id_withouttopics), 'check_topics'] = 0
    #Combine both results
    sub_df['check'] =  sub_df['check_text'] + sub_df['check_topics']
    sub_df.loc[sub_df['check']>=1, 'check'] = 1
    print "Number of opportunities containing at least one of the words:"
    print len(sub_df[sub_df.check == 1])
    return sub_df

In [9]:
def compute_percentage_per_year(sub_df):
    calc_check = pd.crosstab(index=sub_df["check"], columns=sub_df["Year"],margins=True)
    list_years = list(set(sub_df["Year"]))
    list_years.sort()
    list_years.append("rowtotal")
    calc_check.columns = list_years
    calc_check.index= ["False", "True","coltotal"]
    new_calc_check=  (calc_check/calc_check.ix["coltotal"])*100
    return new_calc_check

##### Functions to analyse competitors

In [10]:
def build_corresp_table_competitors(df):
    list_competitors = list(df.Competitors)
    dataframe = pd.DataFrame(list_competitors)
    dataframe['Id'] = pd.DataFrame(df.Id)
    Opp_competitors = pd.melt(dataframe,id_vars='Id').sort('Id')
    Opp_competitors = Opp_competitors[ pd.notnull(Opp_competitors['value'])] #Remove None
    Opp_competitors = Opp_competitors[Opp_competitors['value'] != ''] #Remove empty cells
    Opp_competitors = Opp_competitors[['Id','value']]
    Opp_competitors.columns = ['Id', 'Competitor']
    Opp_competitors = Opp_competitors.reset_index(drop = True)
    #Add Open Date variable to the correspondence table
    Opp_competitors = pd.merge(Opp_competitors, df,left_on='Id', right_on= 'Id')[['Id', 'Competitor', 'Open_Date']]
    Opp_competitors[ pd.isnull(Opp_competitors['Open_Date'])] #Remove Opportunities without open dates
    Opp_competitors['Year']= Opp_competitors['Open_Date'].map(lambda X: X.year) #Create an "Open Year" variable
    return Opp_competitors

In [11]:
#Function to remove wrong competitors
def remove_wrong_competitors(Opp_competitors, list_wrong_competitors):
    for wrong_comp in list_wrong_competitors:
        Opp_competitors = Opp_competitors[Opp_competitors.Competitor != wrong_comp]
    return Opp_competitors

#Function to change all competitor names containing a word to this word (ex: Google Gmail becomes Google)
def clean_main_competitors(Opp_competitors, list_competitors_to_clean):
    for comp in list_competitors_to_clean:
        Opp_competitors.loc[Opp_competitors.Competitor.apply(lambda X: comp in X), 'Competitor'] = comp
    return Opp_competitors

In [12]:
#Function to count the number of mentions of each competitor per Year (and the percentage of mentions per Year)
def count_competitors_per_year(Opp_competitors):
    count_competitors_date = pd.DataFrame(Opp_competitors.groupby(['Year','Competitor'])['Id'].nunique())
    count_competitors_date = count_competitors_date.reset_index()
    percentage_computation = count_competitors_date.groupby('Year').Id.apply(lambda x: 100*x/float(x.sum()))
    count_competitors_date['pct'] = percentage_computation
    count_competitors_date = count_competitors_date.sort_values( by = 'Year', ascending = True).reset_index(drop = True)     
    return count_competitors_date

#Function to add counts equal to 0 if the competitor is not mentioned at all
def add_zeros(count_competitors_date):
    list_displayed_competitors = list(set(count_competitors_date.Competitor))
    list_years = list(set(count_competitors_date.Year))
    list_years.sort()
    #Put 0 in empty quarters / years
    for competitor in list_displayed_competitors:
        sub_df_comp = count_competitors_date[count_competitors_date.Competitor == competitor]
        existing_dates = list(set(sub_df_comp.Year))
        missing_dates = [x for x in list_years if x not in existing_dates]
        if len( missing_dates) > 0:
            to_append = pd.DataFrame(missing_dates)
            to_append['Competitor'] = competitor
            to_append['Id'] = 0
            to_append['pct'] = 0
            to_append.columns = ['Year', 'Competitor', 'Id', 'pct']
            count_competitors_date = count_competitors_date.append(to_append)
    
    count_competitors_date = count_competitors_date.sort_values( by = 'Year', ascending = True).reset_index(drop = True)
    return count_competitors_date

# Import data and preprocess

In [13]:
#################################################################################################################################
sf_username = ""
sf_password = ""
sf_api_token = ""    
#################################################################################################################################

#Extract data from salesforce
df = extract_data_from_sf(sf_username, sf_password, sf_api_token)
topics_df =  extract_topics_from_sf(sf_username, sf_password, sf_api_token)
#Process competitors
df = process_competitors_and_textvariables(df)

#Store the result as Pickle file

#################################################################################################################################
os.chdir('C:/Users/')
#################################################################################################################################

import pickle

file_Name = "df.p"
# open the file for writing
fileObject = open(file_Name,'wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(df,fileObject)   

# here we close the fileObject
fileObject.close()


file_Name_2 = "topics_df.p"
# open the file for writing
fileObject_2 = open(file_Name_2,'wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(topics_df,fileObject_2)   

# here we close the fileObject
fileObject.close()

Number of extracted opportunities:
8893


C:\Users\vtec-svtec1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
df.head(2)

# Load preprocessed data

In [13]:
#################################################################################################################################
os.chdir('C:/Users/')
#################################################################################################################################

#Load processed df
import pickle
file_Name = "df.p"
# we open the file for reading
fileObject = open(file_Name,'r')  
# load the object from the file into var b
df = pickle.load(fileObject)  

file_Name_2 = "topics_df.p"
# we open the file for reading
fileObject_2 = open(file_Name_2,'r')  
# load the object from the file into var b
topics_df = pickle.load(fileObject_2)  

# Check which opportunities contain certain words and plot/export results

Examples:

**E-commerce** : list_words_to_find  = ["e-commerce","ecommerce","e commerce","e-merchant"]
             list_words_to_avoid = ["saas", "software"]

**Crowdfunding / P2P Lending** : ['crowdfunding', "crowd funding", "p2p lending", "peer to peer lending", "peer-to-peer lending", "ptop lending", "crowdequity", "crowd equity", "money lending"]

**Insuretech** : list_words_to_find = ['insurance', 'insuretech', 'insurtech', 'insure tech', 'insurance tech']
list_words_to_find_only_in_topics = []
list_words_to_avoid = ['p2p boat renting platform', 'home exchange', 'neighborhood life', 'freelancers', 'bug bounty']

**Blockchain** : ['blockchain', 'bitcoin']

**Robo-Advisors** : ['robo-advisor', 'robo advisor', 'roboadvisor', 'investment management', 'wealth management', 'finance management', 'savings management', 'financial risk management']

**AI / Big Data / Data Science** : list_words_to_find = [' ai ', 'artificial intelligence', 'machine learning', 'data science', 'predictive algorithms', 'predictive analysis', 'predictive analytics', 'data analytics', 'data mining', 'big data']

**Food** : 
list_words_to_find = ['food', 'foodtech', 'food delivery', 'grocery delivery', 'meals']
list_words_to_avoid = ['pets', 'animals']

**SaaS** : list_words_to_find = ['saas']

**Drones** : list_words_to_find = ['drone']

**Cars** :  ['car sharing', 'carsharing', 'car rental', 'cars', 'vehicles', 'telematics', 'car selling', 'car pooling', 'carpooling']

**Education** : list_words_to_find = ['e-learning', 'elearning', 'online course', 'online lesson', 'online class', 'online learning', 'mooc','edtech', 
                      'education platform', 'teachers', 'teaching', 'educational', 'educative', 'tutoring', 'corporate training', 
                      'corporate education', 'professional training', 'academic', 'education']
list_words_to_find_only_in_topics = ['education']
list_words_to_avoid = ['real estate']

**Smart Home** : list_words_to_find = ['smart home', 'connected home','smarthome', 'smart light', 'connected light' 'connected thermostat', 'smart lock', 'smartlock', 'smart thermostat', 'smart door', 'connected door',
                     'connected kitchen', 'connected house']

**Virtual Assistant** :  list_words_to_find = ['virtual assistant', 'online assistant', 'chatbot', 'chat bot', 'personal assistant', ' bot ', 'chat assistant']

**Agriculture**: list_words_to_find = ['agriculture', 'farming', 'farmer']

In [ ]:
#Check which opportunities contain certain words in Summary or Description or Market Competitors
#################################################################################################################################
year_min = 2012
list_words_to_find = ['crowd sourc', 'crowdsourc']
list_words_to_find_only_in_topics = []
list_words_to_avoid = []
#################################################################################################################################

#Without looking for the words in the 'Business Model / Figures' text
sub_df = findwordfrequency_withoutBM(df, topics_df, year_min, list_words_to_find, list_words_to_find_only_in_topics, list_words_to_avoid)

#Looking for the words everywhere
#sub_df = findwordfrequency_withBM(df, year_min, list_words_to_find, list_words_to_avoid)

############## CHECK IF THE SELECTED OPPORTUNITIES MAKE SENSE !!!!! ############################################################
print 'Opportunities containing at least one of the word:'
sub_df[sub_df.check == 1][['Name', 'Summary', 'Description', 'Competitors']]

In [21]:
#Export to check precisely
sub_df[sub_df.check == 1][['Name', 'Summary', 'Description', 'Competitors']].to_csv('results.csv', index=True)

In [19]:
#Compute percentages per year
percentage_table = compute_percentage_per_year(sub_df)
percentage_table

,2012,2013,2014,2015,2016,rowtotal
False,99.566161,99.18284,99.072513,98.505338,99.523356,99.115683
True,0.433839,0.81716,0.927487,1.494662,0.476644,0.884317
coltotal,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000


In [23]:
#Export results to csv
percentage_table.to_csv('results.csv', index=True)

In [20]:
#Plot results in Plotly
######################################## CHOOSE YEARS TO KEEP ###################################################################
percentage_table = percentage_table[[2012,2013,2014,2015, 2016]]
#################################################################################################################################
import plotly
from plotly.graph_objs import Scatter
from plotly.graph_objs import *

plotly.offline.plot({
    'data': [
        Scatter(x=percentage_table.columns,
                y=list(percentage_table.ix[1]),
                text='',
                marker=Marker(),
                mode='lines+markers',
                line=dict(shape='spline'))
    ],
    'layout': Layout(xaxis=XAxis(title='Time'), yaxis=YAxis(title='Percentage of yearly dealflow'), 
                     title = 'Evolution of the percentage of SaaS opportunities')
}, show_link=True)

'file://C:\\Users\\vtec-svtec1\\Desktop\\Dealflow text mining\\temp-plot.html'

# Analyse competitors

In [ ]:
#Build a dataframe containing 1 line for each Opportunity/Competitor pair
Opp_competitors = build_corresp_table_competitors(df)

################################################################################################################################
list_wrong_competitors = ['us', 'etc', 'etc.', 'direct', 'indirect', 'traditionalplayers', '...', 'pureplayers', 'france', 'germany']
#################################################################################################################################
#Remove names identified as "wrong competitors"
Opp_competitors = remove_wrong_competitors(Opp_competitors, list_wrong_competitors)


#################################################################################################################################
list_competitors_to_clean = ['facebook', 'google', 'youtube', 'oracle', 'microsoft', 'ibm', 'airbnb', 'groupon', 'linkedin', 
                             'ebay', 'apple', 'amazon', 'netflix', 'dropbox', 'leboncoin', 'salesforce', 'yelp', 'tripadvisor',
                            'uber', 'paypal', 'skype', 'withings', 'meetic', 'tinder', 'foursquare', 'deezer', 'viber','expedia',
                            'asos', 'farfetch', 'wix', 'prestashop','sony', 'twitter', 'instagram', 'pinterest', 'adobe', 'whatsapp',
                            'snapchat', 'cisco', 'zynga', 'rovio', 'zalando', 'kickstarter', 'slack', 'parrot', 'wordpress','magento',
                            'lafourchette', 'pagesjaunes', 'stripe']
#################################################################################################################################
############# /!\ ALWAYS CHECK THE RESULT BEFORE ADDING ANY NEW NAME TO THE LIST (Some companies 
#             have names contained in other well known companies, especially companies with abbreviations in their names) ###########
Opp_competitors = clean_main_competitors(Opp_competitors, list_competitors_to_clean)


########################################################### CUSTOM CHANGES #########################################################
Opp_competitors.Competitor[Opp_competitors.Competitor.apply(lambda X: 'ventepriv' in X)] = 'vente-privee'
Opp_competitors.Competitor[Opp_competitors.Competitor.apply(lambda X: 'vente-priv' in X)] = 'vente-privee'
Opp_competitors.Competitor[Opp_competitors.Competitor.apply(lambda X: 'vestiaire' in X)] = 'vestiairecollective'
Opp_competitors.Competitor[Opp_competitors.Competitor.apply(lambda X: 'windows' in X)] = 'microsoft'
Opp_competitors.Competitor[Opp_competitors.Competitor.apply(lambda X: 'itunes' in X)] = 'apple'


Opp_competitors.head()

In [ ]:
#Count mentions of each competitor per year (and the percentage among all mentions per year)
count_competitors_date = count_competitors_per_year(Opp_competitors)

##########################################################################################################################################
year_min = 2013
##########################################################################################################################################

#Filter by year
count_competitors_date = count_competitors_date[count_competitors_date.Year >= year_min]

#Check which are the most cited competitors among all opportunities since year_min (See competitors cited more than nb_mentions_min times)
nb_mentions_min = 5
main_comp_df = pd.DataFrame(count_competitors_date.groupby('Competitor')['Id'].sum()).sort_values(by='Id', ascending = False).reset_index()
main_comp_df = main_comp_df[main_comp_df.Id >= nb_mentions_min]
#Display the most cited competitors among all opportunities since year_min
main_comp_df

In [ ]:
#Choose competitors to display on the graph

################################################################################################################################
list_competitors_to_display = ['airbnb', 'booking.com', 'tripadvisor', 'yelp', 'expedia', 'couchsurfing']
################################################################################################################################

count_competitors_date_to_display = count_competitors_date[count_competitors_date.Competitor.isin(list_competitors_to_display)]
count_competitors_date_to_display = add_zeros(count_competitors_date_to_display)
count_competitors_date_to_display.head()

In [ ]:
#Export counts & pct for selected competitors to csv
count_competitors_date_to_display.to_csv('results_competitors.csv', index=True)

In [24]:
#Plot
import plotly
from plotly.graph_objs import Scatter
from plotly.graph_objs import *

plotly.offline.plot({
    'data': [
        Scatter(x=count_competitors_date_to_display[count_competitors_date_to_display.Competitor == competitor].Year,
                y=count_competitors_date_to_display[count_competitors_date_to_display.Competitor == competitor].pct,
                text='',
                marker=Marker(),
                mode='lines+markers',
                line=dict(shape='spline'),
                name=competitor) for competitor in list_competitors_to_display
    ],
    'layout': Layout(xaxis=XAxis(title='Time'), yaxis=YAxis(title='Percentage of yearly dealflow'), title = 'Startups main competitors')
}, show_link=True)

'file://C:\\Users\\vtec-svtec1\\Desktop\\Dealflow text mining\\temp-plot.html'